In [27]:
from google.colab import files
uploaded = files.upload()


Saving cmn.txt to cmn (2).txt


In [0]:
import re
import jieba
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pickle


In [0]:
batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 2000  # Number of samples to train on.
data_path = 'cmn (2).txt'


In [0]:
def clean_en(x):
  x = x.lower()
  x = re.sub('[,.!?]', '', x)
  return x

def clean_zh(x):
  x = re.sub('[,.!?。，！？\n\t]', '', x)
  # use jieba parser to cut
  x = jieba.cut(x)
  return ' '.join(x)


In [52]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(clean_en(input_text))
    target_texts.append(clean_zh(target_text))
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2000
Number of unique input tokens: 67
Number of unique output tokens: 1274
Max sequence length for inputs: 17
Max sequence length for outputs: 19


In [53]:
print(target_texts)

['嗨', '你好', '你 用 跑 的', '等等', '你好', '让 我 来', '我 赢 了', '不会 吧', '乾杯', '你 懂 了 吗', '他 跑 了', '跳进来', '我 迷失 了', '我 退出', '我 沒事', '听 着', '不 可能', '没门', '你 确定', '试试 吧', '我们 来 试试', '为什么 是 我', '去 问 汤姆', '好棒', '冷静 点', '公平 点', '友善 点', '和 气点', '联系 我', '联系 我们', '进来', '找到 汤姆', '滾 出去', '出去', '走開', '滾', '走開', '再见', '告辞', '等 一下', '他来 了', '他 跑', '帮 我 一下', '帮帮 我们 吧', '坚持', '抱抱 汤姆', '我 同意', '我 生病 了', '我 老 了', '没关系', '是 我', '来 加入 我们 吧', '留着 吧', '吻 我', '完美', '再见', '閉嘴', '不管 它', '拿走 吧', '醒醒', '去 清洗 一下', '我们 知道', '欢迎', '谁 赢 了', '为什么 不', '你 跑', '往 后退 点', '静静的 别动', '我 一无所知', '把 他 铐 上', '往前 开', '走開', '滾', '趴下', '滾', '醒醒 吧', '做得好', '干 的 好', '抓住 汤姆', '抓住 他', '玩得 開心', '他来 试试', '你 就 随 了 我 的 意 吧', '趕快', '快点', '我 忘 了', '我 放弃', '我來 付錢', '我 很 忙', '我 冷', '我 很 好', '我 吃 飽 了', '我 生病 了', '我病 了', '我 个子 高', '让 我 一个 人 呆 会儿', '走 吧', '我們 開始 吧', '我們 走 吧', '当心', '她 跑', '起立', '他们 赢 了', '汤姆 去世 了', '汤姆 不干 了', '汤姆 游泳 了', '相信 我', '努力', '试试 吧', '谁 死 了', '鳥類 飛行', '打电话 回家', '抓住 他', '回家吧', '現在 就 做', '狗会 叫', '别哭', '对不起', '好棒', '来 感受一下 这个', '请 跟我来

In [55]:
print(target_characters)

['\t', '\n', ' ', '!', '"', ',', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'C', 'D', 'J', 'T', 'e', 'i', 'm', 'o', 'r', 't', 'w', '“', '”', '。', '一', '七', '万', '三', '上', '下', '不', '丐', '世', '东', '丟', '丢', '两', '严', '个', '中', '为', '主', '丽', '举', '久', '么', '义', '之', '乎', '乏', '乐', '乞', '也', '习', '乡', '书', '买', '乾', '了', '争', '事', '云', '五', '井', '亚', '些', '亡', '交', '产', '亮', '亲', '人', '什', '今', '仍', '从', '仔', '他', '付', '令', '以', '们', '件', '任', '份', '企', '休', '优', '伙', '会', '伤', '伯', '伴', '位', '住', '佑', '何', '余', '作', '你', '佬', '使', '來', '依', '侦', '侧', '便', '係', '保', '信', '個', '們', '倒', '倖', '候', '借', '倦', '假', '做', '停', '偷', '備', '傷', '傻', '像', '僱', '價', '儿', '允', '兄', '光', '克', '免', '兒', '入', '內', '全', '兩', '公', '六', '共', '关', '兴', '典', '再', '冒', '写', '冬', '冰', '决', '冷', '冻', '净', '准', '凉', '减', '几', '凶', '出', '击', '分', '切', '刚', '別', '利', '别', '到', '則', '前', '剛', '剧', '剪', '劃', '力', '办', '功', '加', '动', '助', '努', '勇', '動', '包', '匆', '匈', '化', '匙', '医', '十', '升', '午', '半', '单

In [54]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1600 samples, validate on 400 samples
Epoch 1/20





1600/1600 [==============================] - 7s 4ms/step - loss: 2.2717 - acc: 0.7713 - val_loss: 1.8501 - val_acc: 0.7592
Epoch 2/20
1600/1600 [==============================] - 1s 893us/step - loss: 1.4864 - acc: 0.8037 - val_loss: 1.8190 - val_acc: 0.7592
Epoch 3/20
1600/1600 [==============================] - 1s 928us/step - loss: 1.4598 - acc: 0.8037 - val_loss: 1.8287 - val_acc: 0.7592
Epoch 4/20
1600/1600 [==============================] - 2s 948us/step - loss: 1.4350 - acc: 0.8037 - val_loss: 1.8440 - val_acc: 0.7592
Epoch 5/20
1600/1600 [==============================] - 1s 933us/step - loss: 1.4203 - acc: 0.8037 - val_loss: 1.8373 - val_acc: 0.7592
Epoch 6/20
1600/1600 [==============================] - 2s 939us/step - loss: 1.4068 - acc: 0.8037 - val_loss: 1.8414 - val_acc: 0.7592
Epoch 7/20
1600/1600 [==========